In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

/home/pranjut/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
# print(housing.data)
# np.c_[np.ones((m, 1)), housing.data]

In [3]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "X")
Y = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "Y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y)


In [4]:
with tf.Session() as sess:
    theta_eval = theta.eval()
    
# theta_eval

## Gradient Descent

In [5]:
n_epochs=1000
learning_rate=0.01

In [6]:
x=tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y=tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta=tf.Variable(tf.random_uniform([n+1, 1], -1, 1), name="theta")
y_pred=tf.matmul(x,theta, name="predictions")
error=y_pred-y

mse=tf.reduce_mean(tf.square(error), name="mse")

gradients=tf.gradients(mse, [theta])[0] #2/m * tf.matmul(tf.transpose(x), error)
training_op=tf.assign(theta, theta - learning_rate * gradients)

In [7]:
init=tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval)
        sess.run(training_op)
    
    best_theta=theta.eval()

Epoch 0 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 100 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 200 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 300 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 400 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 500 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 600 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 700 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 800 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>
Epoch 900 MSE= <bound method Tensor.eval of <tf.Tensor 'mse:0' shape=() dtype=float32>>


## Optimizers

### Gradient Optimizer 

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_ops = optimizer.minimize(mse)

### Momentum Optimizer

In [10]:
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
# training_ops = optimizer.minimize(mse)

### Feeding data

In [11]:
A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A: [[1,2,3],[4,5,6]]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 6.  7.  8.]
 [ 9. 10. 11.]]


#### Feeding data with placeholder

To implement Mini-batch Gradient Descent, we only need to tweak the existing code slightly. First change
the definition of X and y in the construction phase to make them placeholder nodes:

In [12]:
x = tf.placeholder(tf.float32, shape=(None, n+1), name="x")
y = tf.placeholder(tf.float32, shape=(None, 1), name = "y")


Then define the batch size and compute the total number of batches:

In [13]:
batch_size = 100
n_batches= int(np.ceil(m/batch_size))

In [14]:
def fetch_batch(epoch, index, size):
    data = housing_data_plus_bias.data
    labels = housing.target
    data_batch=[housing_data_plus_bias[i] for i in range(index-1, (index + size) -1)]
    label_batch=[[housing.target[i]] for i in range(index-1, (index + size) -1)]
    return data_batch, label_batch
# x, y = fetch_batch(1, 0, 10)
# print(x)
# X.shape

In [15]:
# saver = tf.train.Saver()

with tf.Session() as sess:
#     saver.restore(sess, "/tmp/handson.ckpt")
    sess.run(init)
    for epoch in range(n_epochs):
        for index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, index, batch_size)
#             saver.save(sess, "/tmp/handson.ckpt")
            sess.run(training_ops, feed_dict={x: x_batch, y: y_batch})
    best_theta = theta.eval()